In [2]:
import Min_vertex_cover as mvc
import networkx as nx
import rustworkx as rx
import csv
import tensorflow as tf
import matplotlib.pyplot as plt

import pennylane as qml
from pennylane.wires import Wires
from pennylane import numpy as np
from pennylane import qaoa

In [3]:
edges = edges = [(0,i) for i in range(1,10)]
graph = nx.Graph(edges)

In [4]:
print(mvc.bit_flip_mixer(graph,1))
print(mvc.cost_h(graph,1))

  (0.001953125) [X0]
+ (0.5) [X1]
+ (0.5) [X2]
+ (0.5) [X3]
+ (0.5) [X4]
+ (0.5) [X5]
+ (0.5) [X6]
+ (0.5) [X7]
+ (0.5) [X8]
+ (0.5) [X9]
+ (-0.5) [X1 Z0]
+ (-0.5) [X2 Z0]
+ (-0.5) [X3 Z0]
+ (-0.5) [X4 Z0]
+ (-0.5) [X5 Z0]
+ (-0.5) [X6 Z0]
+ (-0.5) [X7 Z0]
+ (-0.5) [X8 Z0]
+ (-0.5) [X9 Z0]
+ (-0.001953125) [X0 Z9]
+ (-0.001953125) [X0 Z8]
+ (-0.001953125) [X0 Z7]
+ (-0.001953125) [X0 Z6]
+ (-0.001953125) [X0 Z5]
+ (-0.001953125) [X0 Z4]
+ (-0.001953125) [X0 Z3]
+ (-0.001953125) [X0 Z2]
+ (-0.001953125) [X0 Z1]
+ (0.001953125) [X0 Z8 Z9]
+ (0.001953125) [X0 Z7 Z9]
+ (0.001953125) [X0 Z7 Z8]
+ (0.001953125) [X0 Z6 Z9]
+ (0.001953125) [X0 Z6 Z8]
+ (0.001953125) [X0 Z6 Z7]
+ (0.001953125) [X0 Z5 Z9]
+ (0.001953125) [X0 Z5 Z8]
+ (0.001953125) [X0 Z5 Z7]
+ (0.001953125) [X0 Z5 Z6]
+ (0.001953125) [X0 Z4 Z9]
+ (0.001953125) [X0 Z4 Z8]
+ (0.001953125) [X0 Z4 Z7]
+ (0.001953125) [X0 Z4 Z6]
+ (0.001953125) [X0 Z4 Z5]
+ (0.001953125) [X0 Z3 Z9]
+ (0.001953125) [X0 Z3 Z8]
+ (0.001953125) [X0 Z3 Z7

In [5]:
print(mvc.commutation_of_mixer_cost(graph))

  (-0.00390625j) [Y0]
+ (-1j) [Y1]
+ (-1j) [Y2]
+ (-1j) [Y3]
+ (-1j) [Y4]
+ (-1j) [Y5]
+ (-1j) [Y6]
+ (-1j) [Y7]
+ (-1j) [Y8]
+ (-1j) [Y9]
+ (0.00390625j) [Y0 Z9]
+ (0.00390625j) [Y0 Z8]
+ (0.00390625j) [Y0 Z7]
+ (0.00390625j) [Y0 Z6]
+ (0.00390625j) [Y0 Z5]
+ (0.00390625j) [Y0 Z4]
+ (0.00390625j) [Y0 Z3]
+ (0.00390625j) [Y0 Z2]
+ (0.00390625j) [Y0 Z1]
+ (1j) [Y1 Z0]
+ (1j) [Y2 Z0]
+ (1j) [Y3 Z0]
+ (1j) [Y4 Z0]
+ (1j) [Y5 Z0]
+ (1j) [Y6 Z0]
+ (1j) [Y7 Z0]
+ (1j) [Y8 Z0]
+ (1j) [Y9 Z0]
+ (-0.00390625j) [Y0 Z8 Z9]
+ (-0.00390625j) [Y0 Z7 Z9]
+ (-0.00390625j) [Y0 Z7 Z8]
+ (-0.00390625j) [Y0 Z6 Z9]
+ (-0.00390625j) [Y0 Z6 Z8]
+ (-0.00390625j) [Y0 Z6 Z7]
+ (-0.00390625j) [Y0 Z5 Z9]
+ (-0.00390625j) [Y0 Z5 Z8]
+ (-0.00390625j) [Y0 Z5 Z7]
+ (-0.00390625j) [Y0 Z5 Z6]
+ (-0.00390625j) [Y0 Z4 Z9]
+ (-0.00390625j) [Y0 Z4 Z8]
+ (-0.00390625j) [Y0 Z4 Z7]
+ (-0.00390625j) [Y0 Z4 Z6]
+ (-0.00390625j) [Y0 Z4 Z5]
+ (-0.00390625j) [Y0 Z3 Z9]
+ (-0.00390625j) [Y0 Z3 Z8]
+ (-0.00390625j) [Y0 Z3 Z7]
+ (-0.

In [6]:
mixer_h = mvc.bit_flip_mixer(graph,1)
cost_h  = mvc.cost_h(graph,1)
commutator_h = mvc.commutation_of_mixer_cost(graph)

mixer_mat = mixer_h.sparse_matrix().real
s = mixer_mat.toarray()[-1]
s = s/np.sqrt(np.sum(s**2))

In [7]:
def qaoa_layer(gamma, beta):
    qaoa.cost_layer(gamma, cost_h)
    qaoa.mixer_layer(beta, mixer_h)

wires = range(len(graph.nodes()))
depth = 1

def circuit(**kwargs):
    for w in wires:
        qml.PauliX(wires=w)

dev = qml.device("default.qubit", wires=wires)

@qml.qnode(dev)
def cost_function():
    circuit()
    return qml.expval(cost_h)

@qml.qnode(dev)
def cost_commutator():
    circuit()
    return qml.expval(H(graph,1))
dev2 = qml.device("default.mixed",wires=wires)
@qml.qnode(dev2)
def cost_mpm():
    qml.RY(2*np.arccos(1/np.sqrt(3)),wires = 0)
    qml.ctrl(qml.Hadamard, control=0)(wires=1)
    qml.ctrl(qml.PauliX, control=1)(wires=2)
    qml.ctrl(qml.PauliX, control=0)(wires=1)
    qml.PauliX(1)
    qml.PauliX(2)
    return qml.expval(cost_h)

In [8]:
print(cost_function())

10.0


In [23]:
print(cost_mpm())

1.0


In [40]:
def H(graph,sign):   
    
    m_coeffs = []
    m_terms = []
    o_coeffs = []
    o_terms = []
    coeffs= []
    terms = []

    graph_nodes = graph.nodes
    # In RX each node is assigned to an integer index starting from 0;
    # thus, we use the following lambda function to get node-values.
    get_nvalue = lambda i: i
    for i in graph_nodes:
        neighbours = list(graph.neighbors(i))
        degree = len(neighbours)

        n_terms = [[qml.PauliX(get_nvalue(i))]] + [
            [qml.Identity(get_nvalue(n)), qml.PauliZ(get_nvalue(n))] for n in neighbours
        ]
        n_coeffs = [[1, -1] for n in neighbours]

        final_terms = [qml.operation.Tensor(*list(m)).prune() for m in itertools.product(*n_terms)]
        final_coeffs = [
            (0.5**degree) * functools.reduce(lambda x, y: x * y, list(m), 1)
            for m in itertools.product(*n_coeffs)
        ]

        m_coeffs.extend(final_coeffs)
        m_terms.extend(final_terms)
    #print(m_coeffs)
    #print(m_terms)
    graph_nodes = graph.nodes()
    c_coeffs = []
    c_terms  = []
    for i in graph_nodes:
        c_coeffs +=[-1]
        c_terms  += [qml.PauliZ(i)]
    #print(c_coeffs)
    #print(c_terms)


    mc_terms = [m_terms,c_terms,m_terms]
    mc_coeffs = [m_coeffs,c_coeffs,m_coeffs]
    final_terms = [qml.operation.Tensor(*list(m)).prune() for m in itertools.product(*mc_terms)]
    final_coeffs = [
            (0.5**degree) * functools.reduce(lambda x, y: x * y, list(m), 1)
            for m in itertools.product(*mc_coeffs)
        ]
    #print(mc_terms)
    coeffs.extend(final_coeffs)
    terms.extend(final_terms)
    return qml.Hamiltonian(coeffs,terms)

In [56]:
print(H(graph,1))

  (-0.125) [X0 Z0 X0]
+ (-0.125) [X2 Z2 X2]
+ (-0.03125) [X1 Z1 X1]
+ (-0.125) [X0 Z0 X2]
+ (-0.125) [X0 Z1 X0]
+ (-0.125) [X0 Z2 X0]
+ (-0.125) [X0 Z2 X2]
+ (-0.125) [X0 Z1 Z0 X0 Z1]
+ (-0.125) [X0 Z1 Z1 X0 Z1]
+ (-0.125) [X2 Z0 X0]
+ (-0.125) [X2 Z0 X2]
+ (-0.125) [X2 Z1 X2]
+ (-0.125) [X2 Z2 X0]
+ (-0.125) [X2 Z1 Z1 X2 Z1]
+ (-0.125) [X2 Z1 Z2 X2 Z1]
+ (-0.0625) [X0 Z0 X1]
+ (-0.0625) [X0 Z1 X1]
+ (-0.0625) [X0 Z1 Z0 X1 Z0]
+ (-0.0625) [X0 Z1 Z1 X1 Z0]
+ (-0.0625) [X1 Z0 X0]
+ (-0.0625) [X1 Z1 X0]
+ (-0.0625) [X1 Z1 X2]
+ (-0.0625) [X1 Z2 X2]
+ (-0.0625) [X1 Z2 Z1 X2 Z1]
+ (-0.0625) [X1 Z2 Z2 X2 Z1]
+ (-0.0625) [X1 Z0 Z0 X0 Z1]
+ (-0.0625) [X1 Z0 Z1 X0 Z1]
+ (-0.0625) [X2 Z1 X1]
+ (-0.0625) [X2 Z2 X1]
+ (-0.0625) [X2 Z1 Z1 X1 Z2]
+ (-0.0625) [X2 Z1 Z2 X1 Z2]
+ (-0.03125) [X1 Z0 X1]
+ (-0.03125) [X1 Z2 X1]
+ (-0.03125) [X1 Z2 Z1 X1 Z2]
+ (-0.03125) [X1 Z2 Z2 X1 Z2]
+ (-0.03125) [X1 Z0 Z0 X1 Z0]
+ (-0.03125) [X1 Z0 Z1 X1 Z0]
+ (0.03125) [X1 Z0 X1 Z0]
+ (0.03125) [X1 Z1 X1 Z2]
+ (0.031

In [18]:
import itertools,functools

In [57]:
qml.pauli.simplify(H(graph,1))

  (-0.125) [X0]
+ (-0.125) [X2]
+ (-0.03125) [X1]
+ (-0.375) [X0 Z2]
+ (-0.375) [Z0 X2]
+ (0.03125) [Z0 X1]
+ (0.03125) [X1 Z2]
+ (0.125) [X0 Z1]
+ (0.125) [Z1 X2]
+ (0.34375) [Z0 X1 X2]
+ (0.375) [X0 X1 Z2]
